# 1. Gyakorlat: Bevezetés a megerősítéses tanulásba

In [70]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

np.random.seed(42)

## Markov láncok

In [71]:
p_s_a = [
    [0.7, 0.2, 0.0, 0.1],  # s0 --> [s1, s2, s3, s4]
    [0.0, 0.0, 0.9, 0.1],  # s1 --> [s1, s2, s3, s4]
    [0.0, 1.0, 0.0, 0.0],  # s2 --> [s1, s2, s3, s4]
    [0.0, 0.0, 0.0, 1.0],  # s3 --> [s1, s2, s3, s4]
]

max_lepes = 50  # Max. lépésszám
s_T = 3  # Terminális állapot

def lanc_futtatasa():
    s_t = 0
    print("Állapotok:", end=" ")
    for _ in range(max_lepes):
        print(f's{s_t}', end=" ")
        if s_t == s_T:  # Terminális állpot esetén kilépés 
            break
        s_t = np.random.choice(range(4), p=p_s_a[s_t])
    else:
        print("...", end="")
    print()

for _ in range(10):
    lanc_futtatasa()

Állapotok: s0 s0 s3 
Állapotok: s0 s1 s2 s1 s2 s1 s2 s1 s2 s1 s3 
Állapotok: s0 s1 s2 s1 s2 s1 s2 s1 s2 s1 s2 s1 s2 s1 s2 s1 s2 s1 s2 s1 s2 s1 s3 
Állapotok: s0 s3 
Állapotok: s0 s1 s2 s1 s2 s1 s2 s1 s2 s1 s2 s1 s2 s1 s2 s1 s3 
Állapotok: s0 s1 s3 
Állapotok: s0 s1 s2 s1 s2 s1 s2 s1 s2 s1 s2 s1 s2 s1 s2 s1 s2 s1 s2 s1 s2 s1 s2 s1 s2 s1 s2 s1 s2 s1 s2 s1 s2 s1 s2 s1 s2 s1 s2 s1 s2 s1 s2 s1 s2 s1 s2 s1 s2 s1 ...
Állapotok: s0 s0 s3 
Állapotok: s0 s0 s0 s1 s2 s1 s2 s1 s3 
Állapotok: s0 s1 s2 s1 s2 s1 s2 s1 s2 s1 s2 s1 s2 s1 s2 s1 s2 s1 s2 s1 s2 s1 s3 


## Markov döntési folyamatok
Például az $s_0$ állapotban, ha az $a_0$ cselekvést választjuk, akkor $0.7$ valószínűséggel az $s_0$ állapotba kerülünk $+10$ jutalommal, $0.3$ valószínűséggel az $s_1$ állapotba kerülünk jutalom nélkül, és soha nem kerülünk az $s_2$ állapotba (tehát az átmeneti valószínűségek $[0.7, 0.3, 0.0]$, a R pedig $[+10, 0, 0]$):

In [72]:
p_s_a = [  # Állapotátmeneti valószínűségek (környezeti dinamika) [s, a, s']
        [[0.7, 0.3, 0.0],  # s0 --> a0 --> [s0, s1, s2]
         [1.0, 0.0, 0.0],  # s0 --> a1 --> [s0, s1, s2]
         [0.8, 0.2, 0.0]], # s0 --> a2 --> [s0, s1, s2]
        [[0.0, 1.0, 0.0],  # s1 --> a0 --> [s0, s1, s2]
         None,             # s1 --> a1 --> [s0, s1, s2]
         [0.0, 0.0, 1.0]], # s1 --> a2 --> [s0, s1, s2]
        [None,             # s2 --> a0 --> [s0, s1, s2]
         [0.8, 0.1, 0.1],  # s2 --> a1 --> [s0, s1, s2]
         None]             # s2 --> a2 --> [s0, s1, s2]
]

R = [  # Jutalmak [s, a, s']
        [[+10, 0, 0],   # s0 --> a0 --> [r0, r1, r2] 
         [0, 0, 0],     # s0 --> a1 --> [r0, r1, r2] 
         [0, 0, 0]],    # s0 --> a2 --> [r0, r1, r2] 
        [[0, 0, 0],     # s1 --> a0 --> [r0, r1, r2] 
         [0, 0, 0],     # s1 --> a1 --> [r0, r1, r2] 
         [0, 0, -50]],  # s1 --> a2 --> [r0, r1, r2] 
        [[0, 0, 0],     # s2 --> a0 --> [r0, r1, r2] 
         [+40, 0, 0],   # s2 --> a1 --> [r0, r1, r2] 
         [0, 0, 0]],    # s2 --> a2 --> [r0, r1, r2] 
]

A = [  # Lehetséges cselekvések halmaza
        [0, 1, 2],  #  s0 --> s0, s1, s2
        [0, 2],     #  s1 --> s0, s2
        [1]         #  s2 --> s1
]

# $V(s)$ állapot-érték
$V_{\pi}(s)=\sum_{a}\pi(a|s)\sum_{s',r}p\left(s',r|s,a\right)\left[r+\gamma v_{\pi}\left(s'\right)\right]$
$\;minden\;s\in S-re$

In [73]:
gamma = 0.90  # Discount factor
max_lepes = 100  # Iterációk maximális száma

# V(s) inicializálása
n_s = len(p_s_a)  # Állapotok száma
V = np.zeros(n_s)

for _ in range(max_lepes):
    V_prev = V.copy()
    for s in range(n_s):
        if A[s] is not None:
            V[s] = np.max([np.sum([p_s_a[s][a][sp] * (R[s][a][sp] + gamma * V_prev[sp]) for sp in range(n_s)]) for a in A[s]])

print("Állapot-érték függvény: ")
print(V)

Állapot-érték függvény: 
[18.91891892  0.         50.13365013]


Ez alapján elmondható, hogy a legjövedelmezőbb állapot az $a_2$.

## $Q(s,a)$ állapot-cselekvés minőség
$Q_{\pi}(s,a)=\sum_{s',r}p\left(s',r|s,a\right)\left[r+\gamma v_{\pi}\left(s'\right)\right]$

In [74]:
gamma = 0.90  # Diszkont ráta: próbáljuk ki különbözp diszkont rátákkal is!

# Q(s,a) inicializálása
Q = np.full((3, 3), -np.inf)  # -np.inf a lehetetlen cselekvésekre
for s, a in enumerate(A):
    Q[s, a] = 0.0   # Minden lehetséges cselekvésre

hist = [] 
for _ in range(max_lepes):
    Q_prev = Q.copy()
    hist.append(Q_prev)
    for s in range(n_s):  # Iteráció minden állapoton
        for a in A[s]:  # Iteráció minden cselekvésen
            Q[s, a] = np.sum([p_s_a[s][a][sp] * (R[s][a][sp] + gamma * np.max(Q_prev[sp])) for sp in range(3)])

hist = np.array(hist)
print('Állapot-cselekvés minőség függvény:')
print(Q, '\n')

Állapot-cselekvés minőség függvény:
[[18.91891892 17.02702703 13.62162162]
 [ 0.                -inf -4.87971488]
 [       -inf 50.13365013        -inf]] 



A MDP esetében az optimális politika, amikor $0.90$-es diszkonttényezőt használ az ügynök az, hogy az $s_0$ állapotban az $a_0$ cselekvést választja, az $s_1$ állapotban az $a_0$ cselekvést választja, és végül az $s_2$ állapotban az $a_1$ cselekvést (az egyetlen lehetséges cselekvést).

## Mohó ügynök
$a_{t}=\underset{a}{argmax}\:Q_{t}(a)$  
A $Q(s,a)$ megadja minden állapothoz tartozóan az ügynök számára leginkább jövedelmező cselekvést. Ez elég arra, hogy fel lehessen állítani egy politikát: $\pi\in{S\rightarrow}A$

In [75]:
def moho_ugynok(s, Q):
    legjobb_A = np.argmax(Q, axis=1)
    return legjobb_A[s]

max_lepes = 100

r_sum = 0
s = 0
for i in range(max_lepes):
    a = moho_ugynok(s, Q)
    sp = np.random.choice(range(n_s), p=p_s_a[s][a])
    r = R[s][a][sp]
    # print(f'i: {i}, s: {s}, a: {a}, r: {r}, s\': {sp}')
    s = sp
    r_sum += r

print(r_sum)

20
